# Volume Reconstruction and Padding

## 3D Reconstruction of 2D Slices of DICOM Images

In [4]:
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict

root_dir_string = "/Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT" # Change as necessary
root_dir_list = sorted(os.listdir(root_dir_string))
data = defaultdict(list)

## Find the max number of slices in order to make all of the inputs consistent through padding
# initialize max number of slices
max_z_dimension = 0

for folder in root_dir_list: 
    
    # Skip at non-data filesp
    if not folder.startswith("PANCREAS"):
        continue

    # Find data files
    # Assuming all data is organized the same
    dcm_files = sorted(list(os.walk(os.path.join(root_dir_string, folder)))[2][2])

    # sets the maximum number of slices
    if len(dcm_files) > max_z_dimension:
        max_z_dimension = len(dcm_files) 

folder_volumes = []

# Go through the files in root directory
for folder in root_dir_list: 
    # leave root_dir_list[:3] when no need to check and just need the finished array
    # change to root_dir_list[:3] to only go through the first 3 folders which is needed when plotting 3D reconstruction for check
    
    # Skip at non-data filesp
    if not folder.startswith("PANCREAS"):
        continue
    
    # all the dicom slices for the folder
    dicom_slices_folder =[]

    # Find data files
    # Assuming all data is organized the same
    data_dir_string = list(os.walk(os.path.join(root_dir_string, folder)))[2][0]
    dcm_files = sorted(list(os.walk(os.path.join(root_dir_string, folder)))[2][2])

    # Read the data
    for image_file in dcm_files:

        cur_data = pydicom.dcmread(os.path.join(data_dir_string, image_file))
        cur_data_pixels = cur_data.pixel_array # extract only the picture
        dicom_slices_folder.append(cur_data)
        data[folder].append(cur_data_pixels) 

    #### start of 3D volume reconstruction
    # Extracting necessary information from the DICOM metadata
    spacing_between_slices = np.abs(dicom_slices_folder[0].ImagePositionPatient[2] - dicom_slices_folder[1].ImagePositionPatient[2])
    rows = dicom_slices_folder[0].Rows
    cols = dicom_slices_folder[0].Columns
    num_slices = len(dicom_slices_folder)

    # Creating an empty 3D array to store the reconstructed volume
    padded_volume = np.zeros((rows, cols, max_z_dimension))

    # Filling the 3D array with DICOM pixel data
    for i, dicom_slice in enumerate(dicom_slices_folder):
        padded_volume[:, :, i] = dicom_slice.pixel_array

    folder_volumes.append((os.path.join(root_dir_string, folder), padded_volume, spacing_between_slices))
    
    print("folder:", folder, "|number of images:", len(dcm_files), "|processed!", sep="", end="\n\n")

### Checking that the 3D reconstruction happened currently
### Only run the following code on a small number of folders
for folder_name, volume, spacing in folder_volumes:
    print("Folder: ", folder_name)
    print("Volume Shape: ", volume.shape)
    print("Spacing: ", spacing)

# visualization of 3D reconstruction with matplotlib
    # Plot the 3D volume
"""    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot voxels
    x, y, z = np.indices(volume.shape) # three mesh grids that correspond to dimensions of 3D volume array
    ax.scatter(x, y, z, c=volume.flatten(), cmap='gray') # scatter plot
    # volume.flatten() determines the color of each point by its corresponding voxel intensity in the volume array

    plt.title(folder_name)
    plt.show()"""



folder:PANCREAS_0001|number of images:240|processed!

folder:PANCREAS_0002|number of images:195|processed!

folder:PANCREAS_0003|number of images:216|processed!

folder:PANCREAS_0004|number of images:221|processed!

folder:PANCREAS_0005|number of images:210|processed!

folder:PANCREAS_0006|number of images:223|processed!

folder:PANCREAS_0007|number of images:201|processed!

folder:PANCREAS_0008|number of images:205|processed!

Folder:  /Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT/PANCREAS_0001
Volume Shape:  (512, 512, 466)
Spacing:  1.0
Folder:  /Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT/PANCREAS_0002
Volume Shape:  (512, 512, 466)
Spacing:  1.0
Folder:  /Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT/PANCREAS_0003
Volume Shape:  (512, 512, 466)
Spacing:  1.0
Folder:  /Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT/PANCREAS_0004
Volume Shape:  (512, 512, 466)
Spacing:  1.0
Folder:  /Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT/PA

"    fig = plt.figure()\n    ax = fig.add_subplot(111, projection='3d')\n\n    # Plot voxels\n    x, y, z = np.indices(volume.shape) # three mesh grids that correspond to dimensions of 3D volume array\n    ax.scatter(x, y, z, c=volume.flatten(), cmap='gray') # scatter plot\n    # volume.flatten() determines the color of each point by its corresponding voxel intensity in the volume array\n\n    plt.title(folder_name)\n    plt.show()"

## Padding the Labels

In [11]:
import os
import nibabel as nib
from collections import defaultdict
import matplotlib.pyplot as plt

root_dir_string = "/Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT" # Change as necessary
root_dir_list = sorted(os.listdir(root_dir_string))
label_dir_string = "/Users/lukeyun/SDS323/TCIA_pancreas_labels-02-05-2017"  # Change as necessary
label_dir_list = sorted(os.listdir(label_dir_string))
data = defaultdict(list)

## Find the max number of slices in order to make all of the inputs consistent through padding
# initialize max number of slices
max_z_dimension = 0

for folder in root_dir_list: 
    
    # Skip at non-data filesp
    if not folder.startswith("PANCREAS"):
        continue

    # Find data files
    # Assuming all data is organized the same
    dcm_files = sorted(list(os.walk(os.path.join(root_dir_string, folder)))[2][2])

    # sets the maximum number of slices
    if len(dcm_files) > max_z_dimension:
        max_z_dimension = len(dcm_files) 

count = 0

# Go through the files in root directory
for filename in label_dir_list:
    if filename.endswith('.nii.gz'):
        # Construct the full path to the file
        filepath = os.path.join(label_dir_string, filename)
        
        # Load the NIfTI file
        img = nib.load(filepath)
        
        # creates the padded image
        padded_img = np.pad(img.get_fdata(), ((0,0), (0,0), (0, max_z_dimension - img.shape[2])), mode = 'constant', constant_values=0)
            # img.get_fdata() : converts NIfTi image info into np array
            # ((0,0), (0,0), (0, max_z_dimension - img.shape[2])) : only pads along the z axis
            # mode = 'constant' : pads with constant values
            # constant_values = 0 : pads with constant value 0

        # Add the padded NIfTI image to the data dictionary
        data[filename].append(img)

        print("Label shape: ", padded_img.shape)
        

Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shape:  (512, 512, 466)
Label shap

In [ ]:
# Open the NIfTI image for visualization
        # Plot the image
        plt.figure(figsize=(8, 8))
        plt.imshow(img.get_fdata()[:, :, img.shape[2] // 2], cmap='gray')
        plt.title(f"NIfTI Image: {filename}")
        plt.axis('off')
        plt.show()